In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import pandas as pd
import os
import random
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [38]:
def func1(x):
    y = x
    while len(y) != 8:
        y = "0" + y
    return y

In [43]:
fp = os.path.join("data","trainInteractions.csv")
data = pd.read_csv(fp)

data['recipe_id'] = data['recipe_id'].apply(lambda x: str(x))
data['recipe_id'] = data['recipe_id'].apply(lambda x: func1(x))
data['user_id'] = data['user_id'].apply(lambda x: str(x))
data['user_id'] = data['user_id'].apply(lambda x: func1(x))

8    500000
Name: user_id, dtype: int64

In [48]:
training = data[0:400000].reset_index(drop = True)
validation = data[400000:].reset_index(drop = True)

In [49]:
#most popular recipes
recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in readCSV(os.path.join("data","trainInteractions.csv.gz")):
    recipeCount[recipe] += 1
    totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

mostPopularRecipes = [i[1] for i in mostPopular]

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalCooked/(3/2): break
    
#ratings for recipes 
recipeRating = defaultdict(set)
for i in list(training.index):
    recipe = training['recipe_id'][i]
    rating = int(training['rating'][i])
    recipeRating[recipe].add(rating)


In [58]:
#jaccard 
usersPerRecipe = defaultdict(set)
recipesPerUser = defaultdict(set)

for i in list(training.index):
    user = training['user_id'][i]
    recipe = training['recipe_id'][i]
    usersPerRecipe[recipe].add(user)
    recipesPerUser[user].add(recipe)

avg = np.mean(list(training['rating']))

def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

def model(i,u):
    similarities = []
    users = usersPerRecipe[i] 
    #most_pop = mostPopularRecipes[:int(l*len(mostPopularRecipes))]
    ratings = list(recipeRating[i])
    for i2 in recipesPerUser[u]:
        if i2 == i: continue
        sim = Jaccard(users, usersPerRecipe[i2])
        similarities.append(sim)
    if len(similarities) == 0: #cold start 
        if i in return1:
            return 1
            #if np.mean(ratings) >= avg:
                #return 1
            #else:
                #return 0
        else:
            return 0
    if max(similarities) < 0.4:
        if i in return1:
            if np.mean(ratings) >= avg - 2.5:
                return 1
            else: 
                return 0
        else:
            return 0
    if max(similarities) < 0.5:
        if i in return1:
            if np.mean(ratings) >= avg - 2:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        if i in return1:
            if np.mean(ratings) >= avg - 1.5:
                return 1
            else:
                return 0
        else:
            return 0

In [59]:
test = validation[['user_id','recipe_id']]
test['actual'] = [1 for i in range(len(test['recipe_id']))]
model_preds = []
for i in list(test.index):
    user = test['user_id'][i]
    recipe = test['recipe_id'][i]
    #model_preds.append(model(recipe, user, 0.01, 0.4))
    model_preds.append(model(recipe, user))
   # print(max(model(recipe, user, 0.66)))

test['prediction'] = model_preds

test

#test['prediction'] = model_preds
#test



/tmp/ipykernel_3665/2950711030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['actual'] = [1 for i in range(len(test['recipe_id']))]


,user_id,recipe_id,actual,prediction
0,90764166,01768679,1,1
1,68112239,24923981,1,1
2,32173358,57597698,1,0
3,30893740,16266088,1,1
4,69780905,62953151,1,1
...,...,...,...,...
99995,49948579,91427071,1,1
99996,37419956,93818327,1,1
99997,35102479,65083672,1,0
99998,53665875,37059341,1,0


In [60]:
def accuracy_func(preds):
    TP = preds[(preds['actual'] == 1) & (preds['prediction'] == 1)].shape[0]
    TN = preds[(preds['actual'] == 0) & (preds['prediction'] == 0)].shape[0]
    FP = preds[(preds['actual'] == 0) & (preds['prediction'] == 1)].shape[0]
    FN = preds[(preds['actual'] == 1) & (preds['prediction'] == 0)].shape[0]
    P = TP + FN
    N = TN + FP
    accuracy = ((TP + TN)/(P + N))*100
    return accuracy

accuracy_func(test)

66.125